# 네이버와 쿠팡 토픽분석을 통한 토픽 및 명사 추출을 이용한 스타일 및 어휘분석

In [1]:
# 필요한 패키지 불러오기
import pandas as pd
from konlpy.tag import Okt 
okt = Okt()
import gensim
import gensim.corpora as corpora
import os
import pyLDAvis.gensim

In [10]:
# 저장한 데이터 불러오기
df = pd.read_excel('(0)naver_reviews.xlsx')
df.head(20)

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1
5,2,비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적...,0
6,1,주문을 11월6에 시켰는데 11월16일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화...,0
7,2,넉넉한 길이로 주문했는데도 안 맞네요 별로예요,0
8,2,보폴이 계속 때처럼 나오다가 지금은 안나네요~,0
9,2,110인데 전문속옷브랜드 위생팬티105보다 작은듯해요. 불편해요. 밴딩부분이 다 신...,0


In [3]:
review = df['reviews']

# 빈 리스트를 만들어 명사 토큰을 추출해 저장한다.
review_noun_tk = []
for n in review:
    review_noun_tk.append(okt.nouns(n)) # 명사 형태소만 출력

#한글자로만 추출된 데이터를 제거하기
review_noun_tk2 = []
for d in review_noun_tk:
    item = [i for i in d if len(i) > 1] # 뽑아낸 토큰 중 크기가 1 이상만 추출
    review_noun_tk2.append(item)

In [4]:
dict = corpora.Dictionary(review_noun_tk2)

In [5]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

# 딕셔너리화 성공~
print(dict.token2id)

{'배공': 0, '엉망': 1, '택배': 2, '가격': 3, '가성': 4, '구매': 5, '대박': 6, '바느질': 7, '바지': 8, '아주': 9, '정말': 10, '조금': 11, '최고': 12, '누락': 13, '다시': 14, '당황': 15, '머그컵': 16, '바로': 17, '배송': 18, '상품': 19, '생각': 20, '선물': 21, '전달': 22, '큰일': 23, '확인': 24, '도로': 25, '민트': 26, '사용': 27, '색상': 28, '손잡이': 29, '계란': 30, '별로': 31, '보기': 32, '실용': 33, '완전': 34, '진짜': 35, '코팅': 36, '여기': 37, '연락': 38, '전화': 39, '주문': 40, '추천': 41, '회사': 42, '길이': 43, '계속': 44, '보폴': 45, '안나': 46, '지금': 47, '밴딩': 48, '밴딩부분': 49, '브랜드': 50, '빅사': 51, '속옷': 52, '신축': 53, '실로': 54, '위생': 55, '이즈': 56, '일반': 57, '전문': 58, '팬티': 59, '사이즈': 60, '베이지': 61, '살색': 62, '약간': 63, '그냥': 64, '컬러': 65, '화면': 66, '별루': 67, '탛캏타': 68, '무게': 69, '촉감': 70, '친구': 71, '실내': 72, '하나': 73, '가게': 74, '대신': 75, '사장': 76, '운영': 77, '고정': 78, '린데': 79, '바퀴': 80, '거품': 81, '듯해': 82, '생기': 83, '정력': 84, '화장': 85, '남자': 86, '성분': 87, '유해': 88, '초등': 89, '겨우': 90, '결착': 91, '모서리': 92, '부분': 93, '불량': 94, '이슈': 95, '채집': 96, '최소한': 97, '커터칼': 98, '테스트': 

In [13]:
# 단어의 빈도수를 확인하기 위해 doc2bow 로 변환한다.
# BoW 모델은 문서의 단어의 빈도수를 표현해준다.
corpus = [dict.doc2bow(word) for word in review_noun_tk2]

In [7]:
# 토픽의 갯수 k 개 생성
k = 3

#gensim 라이브러리를 사용해 lda 모델을 생성
#corpus lda모델
#iterations 각 문서의 주제 할당을 반복하는 횟수
#num_topics 는 위에 지정한 k 값으로 토픽의 갯수
#id2word 각 토큰의 id에 해당하는 토큰을 연결해준다
#passes는 앞서 넣은 corpus를 학습 반복하는 횟수이다 수가 높아지면 더 잘 학습하지만 그만큼 시간이 증가한다.
#workers 쓰레드의 수 (작업자의 수)이다. 보통은 본인의 컴퓨터에 맞춰 설정하게 된다.
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations = 12, num_topics = k, id2word = dict, passes = 5, workers = 10)

In [8]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.087*"배송" + 0.025*"생각" + 0.023*"주문" + 0.022*"별로" + 0.021*"그냥" + 0.020*"포장" + 0.019*"상품" + 0.013*"아주" + 0.008*"물건" + 0.008*"정말" + 0.007*"상태" + 0.007*"바로" + 0.007*"택배" + 0.007*"반품" + 0.007*"설치"'), (1, '0.024*"사이즈" + 0.017*"냄새" + 0.012*"하나" + 0.011*"느낌" + 0.011*"아이" + 0.010*"디자인" + 0.010*"그냥" + 0.010*"진짜" + 0.009*"생각" + 0.009*"색상" + 0.008*"완전" + 0.008*"사진" + 0.007*"부분" + 0.007*"효과" + 0.006*"마음"'), (2, '0.050*"구매" + 0.042*"사용" + 0.034*"가격" + 0.013*"구입" + 0.010*"선물" + 0.009*"조금" + 0.009*"대비" + 0.008*"그냥" + 0.007*"아기" + 0.007*"만족" + 0.007*"정말" + 0.007*"이번" + 0.006*"처음" + 0.006*"추천" + 0.006*"보고"')]


In [9]:
# lda모델을 디스플레이로 출력
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict)
pyLDAvis.display(lda_vis)